<a href="https://colab.research.google.com/github/kch1103/my/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torchvision
import torchvision.transforms as transforms 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat 
from sklearn.preprocessing import LabelBinarizer
from torchvision import models
from torchsummary import summary

In [ ]:
resnet18_pretrained = models.resnet18(pretrained = True)
print(resnet18_pretrained)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
 

In [ ]:
train_raw = loadmat('/content/drive/MyDrive/data/train_32x32.mat')
test_raw = loadmat('/content/drive/MyDrive/data/test_32x32.mat')

In [ ]:
train_images = np.array(train_raw['X'])
test_images = np.array(test_raw['X'])

train_labels = train_raw['y']
test_labels = test_raw['y']

In [ ]:
print(train_images.shape)
print(test_images.shape)

print(train_labels)
print(test_labels)

(32, 32, 3, 73257)
(32, 32, 3, 26032)
[[1]
 [9]
 [2]
 ...
 [1]
 [6]
 [9]]
[[5]
 [2]
 [1]
 ...
 [7]
 [6]
 [7]]


In [ ]:
train_images = np.moveaxis(train_images, -1, 0)
test_images = np.moveaxis(test_images, -1, 0)

print(train_images.shape)
print(test_images.shape)

(73257, 32, 32, 3)
(26032, 32, 32, 3)


In [ ]:
train_images = torch.FloatTensor(train_images)
test_images = torch.FloatTensor(test_images)

In [ ]:
lb = LabelBinarizer()
train_labels = lb.fit_transform(train_labels)
test_labels = lb.fit_transform(test_labels)

train_labels = torch.FloatTensor(train_labels)
test_labels = torch.FloatTensor(test_labels)

In [ ]:
train_images /= 255.0
test_images /= 255.0

learning_rate = 0.001
batch_size = 50
epochs = 100

In [ ]:
train_images = torch.utils.data.DataLoader(dataset = train_images,
                                           batch_size = batch_size,
                                           shuffle = True
                                           )
test_images = torch.utils.data.DataLoader(dataset = test_images,
                                          batch_size = batch_size,
                                          shuffle = True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

In [ ]:
num_classes = 10
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_classes)

resnet18_pretrained.to(device)
model = resnet18_pretrained

In [ ]:
train_images = (torch.Tensor(73257, 3, 32, 32)).to(device)
test_images = (torch.Tensor(26032, 3, 32, 32)).to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(epochs):
  for batch_idx, (data, targets) in enumerate(train_images):
    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0], -1)
    
    #forward 
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad()
    loss.backward()

    #optimizer
    optimizer.step()

ValueError: ignored

In [ ]:
with torch.no_grad():
  prediction = model(test_images)
  accuracy = model.score(test_images, test_labels)
  print('Accuracy :', accuracy)
